In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree

import requests
from lxml import html


import re
import hashlib
import sqlite3
import telegram_send
import telebot
import time
import configparser


from requests.exceptions import ConnectTimeout


In [2]:
#Ограничивает длину слов
WORLD_LIMIT = 400

TEST_CHANEL = '-1002035663893'

In [3]:
# Создание объекта конфигурации
config = configparser.ConfigParser()

# Чтение данных из файла
config.read('config.ini')

BOT_TOKEN = config.get('tokens', 'BOT_TOKEN')

In [4]:
df_chanels = pd.DataFrame([['top_Z_museum', 1], 
                           ['high_Z_museum', 1], 
                           ['region_Z_museum', 1], 
                           ['low_Z_museum', 2],
                           ['gulag', 3],
                                                                                                               
                           ],columns=['category', 'stage'])


d_stage = {
1:'-1002045319076', 
2:'-1002014969214',
3:'-1002124668886',
}

## Функции

In [5]:
def get_div(tree, xpath_format, div_mode=True):
    list_divs = []


    # Выбор элемента
    selected_element = tree.xpath(xpath_format)


    for div in selected_element:
        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(div, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs


# Parse

## Большой лист

In [6]:
dict_mus = {

    
## Профильные/ тематические (мой личный топ Z музеев)
    
'Музей победы': ['https://victorymuseum.ru/playbill/',
'//div[@class="oblique-items"]//div[@class="item"]', 'top_Z_museum'],
    
'Музей заповедник Сталинградская битва': ['https://stalingrad-battle.ru/',
'//div[@class="section gray-bg fringe-top-gray fringe-bottom-gray"]//div[@class="item"]', 'top_Z_museum'],
    
'Музей блокады Ленинграда': ['https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/',
'//*[@class="elementor-post__card"]', 'top_Z_museum'],
    
'Музей минобороны в Москве': ['http://www.cmaf.ru/',
'//*[@class="reg-news"]//p[@class="news-item"]/following-sibling::*[1]', 'top_Z_museum'],
    
'Музей дорогами победы (парк Патриот)': ['https://1418museum.ru/news/',
'//*[@class="newsItem__contains"]', 'top_Z_museum'],
    
'Бородинская битва': ['https://1812panorama.ru/afisha',
'//*[@class="field-more-title"]', 'top_Z_museum'],
    
    
    
    
    
## Столичные чуть менее Z
    
'Музей современной истории России': ['https://sovrhistory.ru/exhibitions/future/index.php',
'//*[@class="exhibitions__text"]', 'high_Z_museum'],
    
'Выставочный центр Манеж, Москва': ['https://moscowmanege.ru/ru/vystavki-all/',
'//*[@class="news-block exhibition"]', 'high_Z_museum'],
    
'Музей Москвы': ['https://mosmuseum.ru/exhibitions/',
'//*[@class="articles-grid"]//*[@class="content"]', 'high_Z_museum'],
    
'ГИМ, Москва': ['https://mediashm.ru/?all=1', '//a[@class="element"]', 'high_Z_museum'],
    
'Музей кино, Москва': ['https://www.museikino.ru/media_center/',
'//div[@class="media__inf"]', 'high_Z_museum'],
    
    
## Региональные Z  

'Тульский музей оружия ': ['https://www.museum-arms.ru/news/?str1=2024',
'//div[@class="newsfirst"]', 'region_Z_museum'],
    
'Музей военной истории «Свердловск: Говорит Москва!» ': ['https://ugvim.ru/news/',
'//a[@class="front_cont h_more_w"]', 'region_Z_museum'],
    
'Дом-музей В.И. Ленина': ['https://www.leninmemorial.ru/news/',
'//div[@class="row row-news-page"]', 'region_Z_museum'],
    
'Мемориал «В честь героев Курской битвы»': ['http://мемориал.бгикм.рф/меню/новости',
'//div[@class="post"]', 'region_Z_museum'],
    
'Мемориальный музей военного и трудового подвига 1941–1945 гг': ['http://patriotrm.clan.su/news/?page1',
'//td[@style="padding:3px;"]', 'region_Z_museum'],
    
'ГОСУДАРСТВЕННЫЙ ВОЕННО-ИСТОРИЧЕСКИЙ МУЗЕЙ-ЗАПОВЕДНИК «ПРОХОРОВСКОЕ ПОЛЕ»': ['https://прохоровское-поле.рф/vystavochnaya-deyatelnost.html',
'//div[@class="news-item"]', 'region_Z_museum'],
    
    
    
    
'Музейно-мемориальный комплекс «История танка Т-34»': ['https://museum-t-34.ru/новости/',
'//div[@class="site_content_left"]', 'region_Z_museum'],
    
'Музей «Боевая слава» г. Дербента': ['https://derbentmuseum.ru/',
'//section[@class="blog"]//div[@class="col-lg-4"]', 'region_Z_museum'],
    
'Музей «Мемориал Победы»': ['https://memorial24.ru/novosti',
'//div[@class="novelty clearfix novelty-teaser"]', 'region_Z_museum'],
    
'Музей «Смоленщина в годы Великой Отечественной войны 1941−1945 гг.': ['https://www.smolensk-museum.ru/novosti/muzeynaya_zhizn/',
'//div[@class="news__item"]', 'region_Z_museum'],
    
'Музей «Невский пятачок': ['https://www.lenoblmus.ru/news',
'//li[@class="col-md-4 col-lg-3 col-sm-6 col-xs-12"]', 'region_Z_museum'],
    
'Музей «Дорога жизни»': ['https://navalmuseum.ru/news',
'//div[@class="content-box"]', 'region_Z_museum'],


## Региональные исторические/краеведческие музеи, в которых что то проскакивает
    
'Самарский областной историко-краеведческий музей им. П.В. Алабина': ['https://alabin.ru/',
'//div[@class="news__post"]', 'low_Z_museum'],
    
'Якутский государственный объединенный музей истории и культуры народов Севера им. Ем. Ярославского': ['https://yakutmuseum.ru/',
'//a[@class="news-item"]', 'low_Z_museum'],
    
'Краснодарский государственный историко-археологический музей-заповедник им. Е.Д. Фелицына': ['https://felicina.ru/news/',
'//div[@class="content-area content-left col-md-8 col-sm-8"]//article', 'low_Z_museum'],
    
'Государственный исторический музей Южного Урала': ['http://www.chelmuseum.ru/news/#content',
'//div[@class="card"]', 'low_Z_museum'],
    
'Тамбовский областной краеведческий музей': ['https://tmbmuseum.ru/mainmenu/news/relevant',
'//h3', 'low_Z_museum'],
    
'Национальный музей Республики Башкортостан ': ['https://museumrb.ru/category/afisha/events/',
'//div[@class="upcoming-event-2 pt-80 pb-80"]//div[@class="single-event single-event-3 mb-30 wow fadeInUp my-single-event-3"]', 'low_Z_museum'],
    
'Ставропольский государственный музей-заповедник': ['https://stavmuseum.ru/',
'//div[@class="anonsiblock"]', 'low_Z_museum'],
    
'Новосибирский государственный краеведческий музей': ['https://youmuseum.ru/news/',
'//div[@class="text"]', 'low_Z_museum'],
    
'Оренбургский губернаторский историко-краеведческий музей ': ['http://ogikm.ru/',
'//div[@class="tab-content"]//div[@class="item"]', 'low_Z_museum'],
    
'Омский государственный историко-краеведческий музей': ['https://sibmuseum.ru/news/',
'//a[@class="news-card__link"]', 'low_Z_museum'],
    
'Национальный музей Удмуртской Республики им. К. Герда': ['https://nmur.ru/afisha',
'//div[@class="single-block"]', 'low_Z_museum'] ,
    
'Саратовский областной музей краеведения': ['http://www.comk.ru/museum/news/',
'//div[@class="news-item"]', 'low_Z_museum'] ,   

'Национальный музей Республики Марий Эл им. Т. Евсеева ': ['https://www.fumus.ru/events/',
'//ul[@class="events__list"]/li', 'low_Z_museum'] ,  
    
'Ростовский областной музей краеведения ': ['https://www.rostovmuseum.ru/calendar/list-of-events/',
'//div[@class="row"]//div[@class="s-news-wrapper"]', 'low_Z_museum'] ,  

'Воронежский областной краеведческий музей ': ['https://museum-vrn.ru/muzej/novosti/',
'//a[@class="i-wrap"]', 'low_Z_museum'] , 
    
'Национальный музей Республики Татарстан ': ['https://tatmuseum.ru/news/?PAGEN_1=4',
'//a[@class="news-item"]', 'low_Z_museum'] , 

'Брянский государственный краеведческий музей ': ['https://bgkm.ru/',
'//div[@class="bloglatestnews1"]/div', 'low_Z_museum'] , 
    
'Музей истории Бурятии им. М.Н. Хангалова ': ['https://muzeyrb.ru/sobytiya-1?page=2',
'//article[@class="col-lg-4 mb-50 animate-conner"]', 'low_Z_museum'] , 
    
'Национальный музей Республики Дагестан им. А. Тахо-Годи ': ['https://dagmuzey.ru/events',
'//a[@class="s-events-wrapper__item s-events-wrapper-item"]', 'low_Z_museum'] , 
    
'Ульяновский областной краеведческий музей имени И.А. Гончарова ': ['https://uokm.site/category/уокм/новости/',
'//a[@class="a news-link"]', 'low_Z_museum'], 
    
'Музей истории города Улан-Удэ ': ['https://uumuseum.ru/news/',
'//a[@class="event-index-item"]', 'low_Z_museum'], 

'Национальный музей Республики Коми ': ['https://museumkomi.ru/?cat=3',
'//div[@class="special-category"]', 'low_Z_museum'], 

'Национальный музей Республики Алтай имени А.В. Анохина ': ['http://musey-anohina.ru/index.php/novosti',
'//div[@class="catItemBody"]', 'low_Z_museum'], 
    
'Чувашский национальный музей ': ['https://chnmuseum.ru/news/',
'//main[@class="site-main"]/article', 'low_Z_museum'],   
    
'Мордовский республиканский объединенный краеведческий музей им. И.Д. Воронина ': ['http://mrkm.ru/novosti/?PAGEN_1=4',
'//a[@class="box news"]', 'low_Z_museum'],  
    
'Национальный музей им. Алдан Маадыр Республики Тыва ': ['https://музейтыва.рф/?cat=16&paged=1',
'//div[@class="main col-lg-9 col-md-8  postlist"]/article', 'low_Z_museum'], 

'Хабаровский краевой музей имени Н.И. Гродекова ': ['https://hkm.ru/news/',
'//a[@class="new-item list-item"]', 'low_Z_museum'], 
    

'Калининградский областной историко-художественный музей ': ['https://koihm.ru/category/novosti/',
'//div[@class="article-inner"]', 'low_Z_museum'], 
    
'Национальный музей Республики Карелия ': ['https://kareliamuseum.ru/news/',
'//article[@class="clearfix l-news-article"]', 'low_Z_museum'], 
    
'Астраханский музей заповедник ': ['https://astmuseum.ru/ru/news/',
'//div[@class="iblock-list-item-wrap  "]', 'low_Z_museum'], 
    
'Сургутский краеведческий музей ': ['https://skmuseum.ru/',
'//ul[@class="is-layout-flow wp-block-post-template has-small-font-size"]/li', 'low_Z_museum'], 

'Магаданский областной краеведческий музей ': ['https://magadanmuseum.ru/index.php?do=cat&category=news',
'//div[@id="dle-content"]/table', 'low_Z_museum'], 

'Курский областной краеведческий музей ': ['http://kursk-museum.ru/',
'//div[@class="clearfix news_content"]', 'low_Z_museum'],  
    
'Национальный музей Республики Адыгея ': ['https://adyg-museum.ru/ru/',
'//div[@class=" col-12 col-sm-12 col-md-6 col-lg-4 col-xl-4"]|//div[@class=" col-12 col-sm-12 col-md-12 col-lg-6 col-xl-6"]', 'low_Z_museum'],  

'Музейно-выставочный центр г. Иваново ': ['http://www.igikm.ru/sobytiya/novosti/',
'//p[@class="announce-text"]', 'low_Z_museum'], 
    
## ГУЛАГ
    
  
'Музей Анны Ахматовой в Фонтанном доме': ['https://www.akhmatova.spb.ru/afisha',
'//a[@class="js-product-link"]', 'gulag'],
    
'Музей Гулага': ['https://gmig.ru/museum/news/',
'//a[@class="news-in-list"]', 'gulag'],
    
'Соловецкий государственный историко-архитектурный и природный музей-заповедник': ['http://solovky.ru/ru/vystavki/v-rossii-peredvizhnye-vystavki',
'//div[@class="field field-name-field-expandable-block field-type-field-collection field-label-hidden"]/div[@class="field-items"]/div', 'gulag'],

'Новый музей обороны и блокады Ленинграда, АНАСТАСИЯ КАРКАЧЕВА О ВЫСТАВКЕ В НОВОМ МУЗЕЕ': ['https://www.colta.ru/articles/art/14086-novyy-muzey-oborony-i-blokady-leningrada',
'//div[@class="mask left_posts"]/article', 'gulag'],
    
'Пермь 36': ['https://mperm36.ru/novosti',
'//div[@class="news-row"]/div', 'gulag'],
    
'Военно-медицинский музей': ['https://milmed.spb.ru/category/новости/',
'//div[@class="col_block"]', 'gulag'],
        
    
    
    
  
                                                   }

In [8]:
from urllib3.exceptions import MaxRetryError



In [7]:
dict_info = {}


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

for name in dict_mus:
    site = dict_mus[name][0]
    xpath = dict_mus[name][1]
    group_name = dict_mus[name][2]
    
    try:
        response = requests.get(site,  verify=False
                                , headers=headers
                               )
    except ConnectTimeout:
        print(f"Время подключения к {site} истекло.")
        continue

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {site}')

C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'victorymuseum.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'stalingrad-battle.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'blokadamus.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\con

Ошибка статуса 403 для https://1418museum.ru/news/


SSLError: HTTPSConnectionPool(host='sovrhistory.ru', port=443): Max retries exceeded with url: /exhibitions/future/index.php (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006)')))

In [ ]:
try:
    # Ваш код, который может вызвать ошибку MaxRetryError
except urllib3.exceptions. as e:
    print(f"Произошла ошибка MaxRetryError: {e}")

In [ ]:
df_raw_p = pd.DataFrame([dict_info]).T

In [ ]:
df_raw_p = df_raw_p.explode(0)
df_raw_p = df_raw_p.reset_index()

In [ ]:
df_raw_p = df_raw_p.rename(columns={'index':'site', 0:'raw_str'})

In [ ]:
df_raw_p[df_raw_p['raw_str'].isna()]

In [ ]:
df_raw_p

In [ ]:
df_raw_p = df_raw_p[df_raw_p['raw_str'].notna()].copy()

In [ ]:
df_raw_p['text_elem'] = df_raw_p['raw_str'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

## Доработка запросов

In [ ]:
target_site = 'https://www.akhmatova.spb.ru/afisha'
temp_s = set()

# Проверяю есть ли вообще такие
for i in dict_mus.values():
    temp_s.add(i[0])
#assert target_site in temp_s




if any(df_raw_p.site == target_site):
    # Оставляю 10 первых значений 
    part_target = df_raw_p[df_raw_p['site']==target_site][:10]
    
    part_all = df_raw_p[df_raw_p['site']!=target_site]
    
    df_raw_p = pd.concat([part_target, part_all]).copy()

In [ ]:
df_raw_p = df_raw_p.reset_index(drop=True).copy()

## Делаем колоку с хэшом

In [ ]:
def generate_hash(input_string):
    # Создаем объект хэша SHA-256
    hash_object = hashlib.sha256()

    # Преобразуем входную строку в байты и обновляем хэш
    hash_object.update(input_string.encode('utf-8'))

    # Получаем шестнадцатеричное представление хэша
    hash_value = hash_object.hexdigest()

    return hash_value

In [ ]:
# Колонка для хеширования
df_raw_p.insert(0,'hash_col', df_raw_p['site']+ ' ' + df_raw_p['text_elem'])

df_raw_p['hash_col'] = df_raw_p['hash_col'].apply(generate_hash)

In [ ]:
df_raw_p['text_elem'] = df_raw_p['text_elem'].str.replace('❾', '')

In [ ]:
df_raw_p = df_raw_p.drop_duplicates(['site', 'text_elem']).copy()

In [ ]:
df_raw_p.insert(0,'date_stamp', pd.to_datetime('today'))

## Загружаем в db

In [ ]:
conn = sqlite3.connect('museums.db')

In [ ]:
# Создаю таблицы

qu = '''
    CREATE TABLE if not EXISTS "parse" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT
)
'''
conn.execute(qu)

qu = '''
    CREATE TABLE if not EXISTS "parse_temp" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT

)
'''
conn.execute(qu)

# Добавляю таблицу, для успешных выполнений
qu = '''
    CREATE TABLE if not EXISTS send_telegram (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE, 
    "chanel_id" TEXT, 
    teleg_mes TEXT
)
'''
conn.execute(qu)

In [ ]:
# Временная таблица
df_raw_p.to_sql('parse_temp', conn, index=False, if_exists='replace')


# Импортирую значения с новыми значениями по хэшу
qu = '''
INSERT INTO parse
select *
from parse_temp
where hash_col not in (select hash_col from parse)

'''
conn.execute(qu)

In [ ]:
# Не загруженые значения
qu = '''
select *
from parse
where hash_col not in (select hash_col from send_telegram)
order by date_stamp
'''
df_n_compl = pd.read_sql(qu, conn)

# Чистка

In [ ]:
def adj_lim(x:list):
    '''Ограничивает длину слова'''
    for i, el  in enumerate(x):

        if len(el) > WORLD_LIMIT:
            x[i] = el[:WORLD_LIMIT]+'...'
            
    return x



list_unuse = ['\t\t\t\tЧитать далее»\t\t\t', 'подробнее >>', 'Подробнее', 'Заказать', 'подробнее...']


def drop_words(x:list):
    '''
    Удаляю лишние слова 
    '''
    new_list = []
    for l in x:

        if not l in list_unuse and not bool(re.match(r'^[0-9 /]+$', l)) and l:
            new_list.append(l)
#             print('********************')
#             print(l)
#             print('********************')
#         else:
#             print('********************')
#             print('Не лобавлено',l)
#             print('********************')
    return new_list

        

In [ ]:
# Удаляю https://vk.com иза ошибки
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(r'Ссылка: https://vk.com[^\\]*', '', regex=True)

In [ ]:
# Убираю \r
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('\r', '')

In [ ]:
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('[0-9]* Comments', '')

In [ ]:
# Сплит строки
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(lambda x: re.split(r"(\n)", x))

# Ограничивает длину
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(adj_lim)

# Удаление лишних слов 
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(drop_words)

df_n_compl['text_elem']= df_n_compl['text_elem'].apply(lambda x: ''.join(x))

In [ ]:
# Удаление \n в конце текста
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('[(\\n)(\\t)]+$', '', regex=True)

# Удаление \n в начале
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('^[(\\n)(\\t)]+', '', regex=True)

# Замена \n\n на меньшее число
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('(\\n){3,}','\n\n', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(' {3,}',' ', regex=True)

# Telegram

## Добавляем имена музеев

In [ ]:
d_mus_names = {}

for k in dict_mus:
    d_mus_names[dict_mus[k][0]] = k

In [ ]:
df_n_compl['mus_name'] = df_n_compl['site'].map(d_mus_names)

df_n_compl['mus_name']= df_n_compl['mus_name'].fillna(df_n_compl['site'])


## Отправка

### Форматирование

### Ссылка на музей

In [ ]:
df_n_compl['teleg_mes'] = '[' + df_n_compl.mus_name + ']' + '(' + df_n_compl.site + ')' + '\n'

df_n_compl['teleg_mes'] = df_n_compl['teleg_mes']+ df_n_compl.text_elem

In [ ]:
df_n_compl = df_n_compl[df_n_compl['text_elem']!='']

In [ ]:
df_n_compl

### Добавляем группы

In [ ]:
df_chanels

In [ ]:
df_chanels['chanel_id'] = df_chanels['stage'].map(d_stage)

In [ ]:
df_chanels

In [ ]:
df_mus_info = pd.DataFrame(dict_mus).T
df_mus_info.columns = ['site', 'xpath', 'category']

In [ ]:
df_mus_info = df_mus_info.merge(df_chanels, right_on=['category'], left_on=['category'], how='left')

In [ ]:
df_mus_info = df_mus_info[['site', 'chanel_id']]

In [ ]:
df_n_compl = df_n_compl.merge(df_mus_info, on='site', how='left')

In [ ]:
# assert df_n_compl[df_n_compl['chanel_id'].isna()].empty


# # Времяно!!!
df_n_compl = df_n_compl[~df_n_compl['chanel_id'].isna()]

In [ ]:
df_n_compl['chanel_id'].value_counts()

In [ ]:
df_mes = df_n_compl[['hash_col', 'teleg_mes', 'chanel_id']].copy()

In [ ]:
df_mes

In [ ]:
df_mes.shape

### Исполнение

In [ ]:
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('_-', '-', regex=True)

df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\*', '', regex=True)

# Убирайю символ не разрывного пробела
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\xa0', ' ', regex=True)

In [ ]:



execute_session = pd.to_datetime('now')


# Замените 'YOUR_BOT_TOKEN' на токен вашего бота
bot = telebot.TeleBot(BOT_TOKEN)



PAUSE_ITER = 20
PAUSE_TIME = 50


# Ожидание перд запуском
# time.sleep(PAUSE_TIME)
for num, tuple_mes in enumerate(df_mes.itertuples()):
    
    try:
        # Отправка сообщения
        bot.send_message(tuple_mes.chanel_id, tuple_mes.teleg_mes, parse_mode='Markdown')
    
        # Сохранение успешных сообщений
        df = pd.DataFrame([tuple_mes]).drop(columns='Index')

        df['date_stamp'] = execute_session

        if num % 19==0 and num!=0:
            print('Ожидание')
            time.sleep(PAUSE_TIME)

        df.to_sql('send_telegram', conn, if_exists='append', index=False)
    except Exception as send_error:
        print(num, send_error, tuple_mes)
        df_error = pd.DataFrame([[tuple_mes.hash_col, num, send_error, 
                    tuple_mes.teleg_mes, tuple_mes.chanel_id]], columns=['hash_col','num', 'error', 'teleg_mes', 'chanel_id'])

        df_error.insert(0,'date_time', execute_session)
        df_error['error'] = df_error['error'].astype('str')
        df_error.to_sql('send_error', conn, index=False, if_exists='append')

    
    


# Тест

In [ ]:
cnjg

In [ ]:
q = df_error['teleg_mes'][0]

In [ ]:
q

In [ ]:
q = '[Музей «Мемориал Победы»](https://memorial24.ru/novosti)\nПриглашаем в Единый Почетный караул\n\n14.10.2023\n\n4-го ноября постовцы советских лет заступят в караул вместе с участниками возрожденного Поста №1. Участие бесплатное, но нужна предварительная регистрация (ссылка ниже )\nМузей «Мемориал Победы» и Пост №1 приглашают тех, кто нес караульную службу в 1968-1992 годы, принять участие в Едином Почетном карауле.\nДвижение юнармейских Постов ведет свою историю с осени 1965-го года, когда в Волгограде был создан первый в стране пионерско-комсомольский Пост №1.\n5-го октября 1968-го года Пост №1 был открыт в Красноярске. До 1992-го года лучшие пионеры и комсомольцы города несли службу у Вечного огня на Красной площади Красноярска. Возрождение Поста №1 и караульной службы у Вечного огня на площади Мемориала Победы стало с февраля 2015-го года.\nЗаявки на участие в едином почётном карауле принимаются до 27-го октября по телефону 201-89-29 или регистрируйтесь здесь: https://vk.com/app6013442_-26051136?form_id=3#form_id=3\n\nПОДЕЛИТЬСЯ:'

In [ ]:
bot.send_message(TEST_CHANEL, q, parse_mode='Markdown')